In [1]:
import torch.nn as nn
import sys
import numpy as np
import torch
from torch.autograd import Variable
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
from collections import OrderedDict
import torch.nn.functional as F
import pandas as pd
from pandas import DataFrame
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

In [2]:
class DenseNet(nn.Module):
    def __init__(self, growth_rate=32, block_config=(6, 12, 24, 16),
                 num_init_features=64, bn_size=4, drop_rate=0, num_classes=4, memory_efficient=False):
 
        super(DenseNet, self).__init__()
 
        # 首层卷积层
        self.Net = nn.Sequential(OrderedDict([
            ('conv0', nn.Conv1d(1, num_init_features, kernel_size=7, stride=2,
                                padding=3, bias=False)),
            ('norm0', nn.BatchNorm1d(num_init_features)),
            ('relu0', nn.ReLU(inplace=True)),
            ('pool0', nn.MaxPool1d(kernel_size=3, stride=2, padding=1)),
        ]))
 
        # 构建DenseBlock
        num_features = num_init_features
        for i, num_layers in enumerate(block_config): #构建4个DenseBlock
            block = _DenseBlock(
                num_layers=num_layers,
                num_input_features=num_features,
                bn_size=bn_size,
                growth_rate=growth_rate,
                drop_rate=drop_rate,
                memory_efficient=memory_efficient
            )
            self.Net.add_module('denseblock%d' % (i + 1), block)
            num_features = num_features + num_layers * growth_rate
            if i != len(block_config) - 1:
                trans = _Transition(num_input_features=num_features,  #每个DenseBlock后跟一个TransitionLayer
                                    num_output_features=num_features // 2)
                self.Net.add_module('transition%d' % (i + 1), trans)
                num_features = num_features // 2
 
        # Final batch norm
        self.Net.add_module('norm5', nn.BatchNorm1d(num_features))
 
        # Linear layer
        self.Classifier = nn.Linear(num_features, num_classes) #构建分类器
 
        # Official init from torch repo.
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.constant_(m.bias, 0)
 
    def forward(self, x):
        features = self.Net(x)
        out = F.relu(features, inplace=True)
        out = F.adaptive_avg_pool1d(out, 1)
#         print(out.shape)
        out = torch.flatten(out, 1)
#         print(out.shape)
        out = self.Classifier(out)
        return out

In [3]:
class _DenseBlock(nn.Module):
    def __init__(self, num_layers, num_input_features, bn_size, growth_rate, drop_rate, memory_efficient=False):
        super(_DenseBlock, self).__init__()
        for i in range(num_layers):
            layer = _DenseLayer(
                num_input_features + i * growth_rate,
                growth_rate=growth_rate,
                bn_size=bn_size,
                drop_rate=drop_rate,
                memory_efficient=memory_efficient,
            )
            self.add_module('denselayer%d' % (i + 1), layer)
 
    def forward(self, init_features):
        features = [init_features]
        for name, layer in self.named_children():
            new_features = layer(*features)
            features.append(new_features)
        return torch.cat(features, 1) #将之前的层拼接在一起，并且按行展开

In [4]:
def _bn_function_factory(norm, relu, conv):
    def bn_function(*inputs):
        concated_features = torch.cat(inputs, 1)
        bottleneck_output = conv(relu(norm(concated_features)))
        return bottleneck_output
    return bn_function

In [5]:
class _DenseLayer(nn.Sequential):
    def __init__(self, num_input_features, growth_rate, bn_size, drop_rate, memory_efficient=False):
        super(_DenseLayer, self).__init__()
        self.add_module('norm1', nn.BatchNorm1d(num_input_features)),
        self.add_module('relu1', nn.ReLU(inplace=True)),
        self.add_module('conv1', nn.Conv1d(num_input_features, bn_size *
                                           growth_rate, kernel_size=1, stride=1,
                                           bias=False)),
        self.add_module('norm2', nn.BatchNorm1d(bn_size * growth_rate)),
        self.add_module('relu2', nn.ReLU(inplace=True)),
        self.add_module('conv2', nn.Conv1d(bn_size * growth_rate, growth_rate,
                                           kernel_size=3, stride=1, padding=1,
                                           bias=False)),
        self.drop_rate = drop_rate
        self.memory_efficient = memory_efficient
 
    def forward(self, *prev_features):
        bn_function = _bn_function_factory(self.norm1, self.relu1, self.conv1)
        if self.memory_efficient and any(prev_feature.requires_grad for prev_feature in prev_features):
            bottleneck_output = cp.checkpoint(bn_function, *prev_features)
        else:
            bottleneck_output = bn_function(*prev_features)
        new_features = self.conv2(self.relu2(self.norm2(bottleneck_output)))
        if self.drop_rate > 0:
            new_features = F.dropout(new_features, p=self.drop_rate,
                                     training=self.training)
        return new_features

In [6]:
class _Transition(nn.Sequential):
    def __init__(self, num_input_features, num_output_features):
        super(_Transition, self).__init__()
        self.add_module('norm', nn.BatchNorm1d(num_input_features))
        self.add_module('relu', nn.ReLU(inplace=True))
        self.add_module('conv', nn.Conv1d(num_input_features, num_output_features,
                                          kernel_size=1, stride=1, bias=False))
        self.add_module('pool', nn.AvgPool1d(kernel_size=2, stride=2)) #尺寸减少一半

In [7]:
class TrainDataset(Dataset):
    def __init__(self, data, label):
        self.data = data
        self.label = label
    
    def __getitem__(self, index):
        data = self.data[index]
        label = self.label[index]
        return torch.Tensor(data),label
    
    def __len__(self):
        return len(self.data)

def train_dataset(train_path):
    train = pd.read_csv(train_path)
    # 处理训练数据
    train_data = []
    train_label = []
    for item in train.values:
        # train_data
        arr = np.array([float(i) for i in item[1].split(',')])
        arr.resize((1,205))
        train_data.append(arr)
        #train_label
#         arr = np.zeros((4))
#         arr[int(item[2])]=1.0
#         train_label.append(arr)
        train_label.append(item[2])
    
    # 分割训练集和验证集
    data = TrainDataset(train_data, train_label)
    train_size = int(len(data) * 0.8)
    validdate_size = int(len(data)) - train_size
    traindata, validdata = torch.utils.data.random_split(data, [train_size, validdate_size])
    return traindata, validdata

In [8]:
class TestDataset(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __getitem__(self, index):
        data = self.data[index]
        return torch.Tensor(data)
    
    def __len__(self):
        return len(self.data)
    
def test_dataset(test_path):
    test = pd.read_csv(test_path)
    # 处理训练数据
    test_data = []
    for item in test.values:
        # train_data
        arr = np.array([float(i) for i in item[1].split(',')])
        arr.resize((1,205))
        test_data.append(arr)
    
    testdata = TestDataset(test_data)
    return testdata

In [9]:
def get_acc(out, label):
    total = out.shape[0]
    _, pred_label = out.max(1)
    num_correct = (pred_label == label).sum().item()
    return num_correct / total

In [10]:
def train(net, trainiter, validiter, num_epochs, optimizer, criterion):
    device = torch.device('cuda:1' if torch.cuda.is_available else 'cpu')
    net = net.to(device)
    print("train start!")
    for epoch in range(num_epochs):
        net = net.train()
        train_loss = 0
        train_acc = 0
        for data, label in trainiter:
#             # 将装有tensor的list转换为tensor
#             data = torch.stack(data,1)
            
            data = data.to(device)
            label = label.to(device, dtype=torch.int64)
            # 前向传播
            out = net(data)
            loss = criterion(out, label)
            # 反向传播
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # 计算loss和accuracy
            train_loss += loss.item()
            train_acc += get_acc(out, label)
        
        if validiter is not None:
            valid_loss = 0
            valid_acc = 0
            net = net.eval()
            for data, label in validiter:
                data = data.to(device)
                label = label.to(device, dtype=torch.int64)
                out = net(data)
                loss = criterion(out, label)
                
                valid_loss += loss.item()
                valid_acc += get_acc(out, label)
                
            print("Epoch %d. Train Loss: %f, Train Acc: %f, Valid Loss: %f, Valid Acc: %f, "
                    % (epoch, train_loss / len(trainiter),train_acc / len(trainiter), 
                       valid_loss / len(validiter),valid_acc / len(validiter)))
        else:
            print("Epoch %d. Train Loss: %f, Train Acc: %f, "
                   %(epoch, train_loss / len(trainiter),train_acc / len(trainiter)))

In [11]:
# train1(学习率为0.1)
if __name__ == '__main__':
    net = DenseNet()   
    batch_size = 1000 
    traindata, validdata = train_dataset('train.csv')
    trainiter = DataLoader(traindata, batch_size=batch_size,shuffle=True)
    validiter = DataLoader(validdata, batch_size=batch_size,shuffle=True)
    num_epochs = 50
    optimizer = torch.optim.SGD(net.parameters(), lr=1e-2) #随机梯度下降
    criterion = nn.CrossEntropyLoss() #loss为交叉熵
    
    train(net, trainiter, validiter, num_epochs, optimizer, criterion)
    torch.save(net, 'net2.pth')

train start!


/home/sgq/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch 0. Train Loss: 0.411726, Train Acc: 0.877500, Valid Loss: 0.160583, Valid Acc: 0.954100, 
Epoch 1. Train Loss: 0.116784, Train Acc: 0.965850, Valid Loss: 0.102337, Valid Acc: 0.971000, 
Epoch 2. Train Loss: 0.079956, Train Acc: 0.976575, Valid Loss: 0.079910, Valid Acc: 0.976700, 
Epoch 3. Train Loss: 0.059991, Train Acc: 0.982112, Valid Loss: 0.069446, Valid Acc: 0.978950, 
Epoch 4. Train Loss: 0.045494, Train Acc: 0.986437, Valid Loss: 0.065891, Valid Acc: 0.981200, 
Epoch 5. Train Loss: 0.037291, Train Acc: 0.988850, Valid Loss: 0.057661, Valid Acc: 0.983650, 
Epoch 6. Train Loss: 0.028208, Train Acc: 0.991675, Valid Loss: 0.056878, Valid Acc: 0.983350, 
Epoch 7. Train Loss: 0.022052, Train Acc: 0.993775, Valid Loss: 0.059002, Valid Acc: 0.984100, 
Epoch 8. Train Loss: 0.017245, Train Acc: 0.995525, Valid Loss: 0.053126, Valid Acc: 0.985450, 
Epoch 9. Train Loss: 0.012162, Train Acc: 0.996987, Valid Loss: 0.054779, Valid Acc: 0.985850, 
Epoch 10. Train Loss: 0.008059, Train Ac

In [11]:
# train2(学习率为0.01)
if __name__ == '__main__':
    net = DenseNet()   
    batch_size = 1000 
    traindata, validdata = train_dataset('train.csv')
    trainiter = DataLoader(traindata, batch_size=batch_size,shuffle=True)
    validiter = DataLoader(validdata, batch_size=batch_size,shuffle=True)
    num_epochs = 100
    optimizer = torch.optim.SGD(net.parameters(), lr=1e-2) #随机梯度下降
    criterion = nn.CrossEntropyLoss() #loss为交叉熵
    
    train(net, trainiter, validiter, num_epochs, optimizer, criterion)
    torch.save(net, 'net2.pth')

train start!


/home/sgq/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch 0. Train Loss: 0.730034, Train Acc: 0.727213, Valid Loss: 0.555673, Valid Acc: 0.828850, 
Epoch 1. Train Loss: 0.473184, Train Acc: 0.874275, Valid Loss: 0.391396, Valid Acc: 0.897800, 
Epoch 2. Train Loss: 0.343598, Train Acc: 0.907863, Valid Loss: 0.295517, Valid Acc: 0.916050, 
Epoch 3. Train Loss: 0.268481, Train Acc: 0.921750, Valid Loss: 0.240232, Valid Acc: 0.928300, 
Epoch 4. Train Loss: 0.221339, Train Acc: 0.935250, Valid Loss: 0.204111, Valid Acc: 0.943150, 
Epoch 5. Train Loss: 0.188543, Train Acc: 0.947062, Valid Loss: 0.178058, Valid Acc: 0.951000, 
Epoch 6. Train Loss: 0.164428, Train Acc: 0.954075, Valid Loss: 0.158812, Valid Acc: 0.956350, 
Epoch 7. Train Loss: 0.146125, Train Acc: 0.959512, Valid Loss: 0.144368, Valid Acc: 0.960600, 
Epoch 8. Train Loss: 0.131908, Train Acc: 0.963175, Valid Loss: 0.132644, Valid Acc: 0.964250, 
Epoch 9. Train Loss: 0.120712, Train Acc: 0.966387, Valid Loss: 0.123240, Valid Acc: 0.966350, 
Epoch 10. Train Loss: 0.111056, Train Ac

Epoch 85. Train Loss: 0.003885, Train Acc: 0.999812, Valid Loss: 0.050616, Valid Acc: 0.986000, 
Epoch 86. Train Loss: 0.003963, Train Acc: 0.999800, Valid Loss: 0.051200, Valid Acc: 0.986400, 
Epoch 87. Train Loss: 0.003729, Train Acc: 0.999850, Valid Loss: 0.051050, Valid Acc: 0.986350, 
Epoch 88. Train Loss: 0.003607, Train Acc: 0.999850, Valid Loss: 0.051454, Valid Acc: 0.986250, 
Epoch 89. Train Loss: 0.003513, Train Acc: 0.999900, Valid Loss: 0.051474, Valid Acc: 0.986250, 
Epoch 90. Train Loss: 0.003498, Train Acc: 0.999875, Valid Loss: 0.051534, Valid Acc: 0.986000, 
Epoch 91. Train Loss: 0.003319, Train Acc: 0.999912, Valid Loss: 0.051506, Valid Acc: 0.985850, 
Epoch 92. Train Loss: 0.003336, Train Acc: 0.999912, Valid Loss: 0.051133, Valid Acc: 0.986100, 
Epoch 93. Train Loss: 0.003351, Train Acc: 0.999875, Valid Loss: 0.051841, Valid Acc: 0.986000, 
Epoch 94. Train Loss: 0.003176, Train Acc: 0.999925, Valid Loss: 0.051304, Valid Acc: 0.985950, 
Epoch 95. Train Loss: 0.003112

In [11]:
# test
if __name__ == '__main__':
    device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
    model = torch.load('net1.pth')
    model = model.to(device)
    model.eval()  # 转为test模式
    batch_size = 1000
    testdata = test_dataset('testA.csv')
    testiter = DataLoader(testdata, batch_size=batch_size,shuffle=False) # 一定要定义为False!
#     print(iter(testiter).next().shape)
    result = []
    for data in testiter:
        data = data.to(device)
        with torch.no_grad():
            out = model(data)
        pre = F.softmax(out, 1)
        pre = pre.to('cpu')
#         print(pre)
        result.append(pre)
    print(result)
    result = torch.stack(result, 0) #按照轴0将list转换为tensor

/home/sgq/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


[tensor([[1.0000e+00, 2.5332e-07, 5.9304e-09, 2.4673e-08],
        [1.6729e-07, 7.2411e-06, 9.9999e-01, 9.4169e-10],
        [4.0069e-08, 1.4034e-06, 8.3287e-08, 1.0000e+00],
        ...,
        [1.5378e-04, 1.7448e-05, 9.9983e-01, 5.2307e-07],
        [1.8216e-06, 9.9998e-01, 1.6172e-05, 3.8846e-06],
        [9.9999e-01, 8.8189e-06, 3.3051e-07, 1.2294e-06]]), tensor([[1.0000e+00, 1.4064e-08, 7.9501e-08, 3.9498e-08],
        [8.9368e-03, 2.0326e-04, 9.9019e-01, 6.7218e-04],
        [9.9968e-01, 3.2479e-04, 5.4699e-08, 1.3502e-07],
        ...,
        [2.4143e-06, 5.1156e-06, 1.3137e-05, 9.9998e-01],
        [9.9215e-01, 1.0123e-04, 7.6869e-03, 6.5770e-05],
        [1.0000e+00, 4.7768e-08, 1.7844e-08, 8.5085e-08]]), tensor([[1.2245e-05, 1.7748e-05, 9.9996e-01, 7.1848e-06],
        [1.4035e-05, 2.6831e-06, 9.9998e-01, 7.2340e-08],
        [1.0000e+00, 2.9090e-07, 2.1095e-09, 2.5298e-08],
        ...,
        [9.0497e-07, 2.3294e-06, 5.3225e-05, 9.9994e-01],
        [5.9161e-05, 1.4291e

In [12]:
# 进行数据的后处理，准备提交数据(设置阈值)
result = np.array(result)
result = result.reshape((20000,4))
thr = [0.7, 0.35, 0.45, 0.5]
for x in result:
    for i in [1, 2, 3, 0]:
        if x[i] > thr[i]:
            x[0:i] = 0
            x[i+1:4] = 0
            x[i] = 1
            break

id =np.arange(100000,120000)
df = DataFrame(result, columns=['label_0','label_1','label_2','label_3'])
df.insert(loc=0, column='id', value=id, allow_duplicates=False) 
df.to_csv("submit.csv", index_label="id", index = False)
df

,id,label_0,label_1,label_2,label_3
0,100000,1.0,0.0,0.0,0.0
1,100001,0.0,0.0,1.0,0.0
2,100002,0.0,0.0,0.0,1.0
3,100003,1.0,0.0,0.0,0.0
4,100004,1.0,0.0,0.0,0.0
...,...,...,...,...,...
19995,119995,1.0,0.0,0.0,0.0
19996,119996,1.0,0.0,0.0,0.0
19997,119997,0.0,0.0,1.0,0.0
19998,119998,1.0,0.0,0.0,0.0


In [13]:
# 进行数据的后处理，准备提交数据(直接取最大值)
result = np.array(result)
result = result.reshape((20000,4))
for x in result:
    for i in range(4):
        if x[i] is max(x):
            x[0:i] = 0
            x[i+1:4] = 0
            x[i] = 1
            break

id =np.arange(100000,120000)
df = DataFrame(result, columns=['label_0','label_1','label_2','label_3'])
df.insert(loc=0, column='id', value=id, allow_duplicates=False) 
df.to_csv("submit1.csv", index_label="id", index = False)
df

,id,label_0,label_1,label_2,label_3
0,100000,1.0,0.0,0.0,0.0
1,100001,0.0,0.0,1.0,0.0
2,100002,0.0,0.0,0.0,1.0
3,100003,1.0,0.0,0.0,0.0
4,100004,1.0,0.0,0.0,0.0
...,...,...,...,...,...
19995,119995,1.0,0.0,0.0,0.0
19996,119996,1.0,0.0,0.0,0.0
19997,119997,0.0,0.0,1.0,0.0
19998,119998,1.0,0.0,0.0,0.0
